In [86]:
import math
import cv2
import numpy as np
import gymnasium as gym

from gymnasium import spaces

In [87]:
from importnb import Notebook
with Notebook():
    from LabCacheService import CacheService

cache = CacheService(capacity_bytes=2_000_000_000)

In [88]:
from importnb import Notebook
with Notebook():
    from LabTrajectory import simulate_viewport_with_tiles, visualize_trajectory_sequence

if __name__ == "__main__":
    n_frames = 1_800
    num_tiles = 4

    yaw, pitch, tiles_per_frame, tiles_array = simulate_viewport_with_tiles(
        num_steps=n_frames,
        n=num_tiles,
        fov_yaw=120,
        fov_pitch=60,
        damping=0.99,
        step_size=1.5,
        start_yaw=180,
        start_pitch=0
    )
    user_tiles = tiles_per_frame[:60]
    # visualize_trajectory_sequence(
    #     yaw, 
    #     pitch, 
    #     user_tiles, 
    #     n=num_tiles, 
    #     fov_yaw=120, 
    #     fov_pitch=60,
    #     start_frame=0, 
    #     num_frames_to_show=60, 
    #     cols=6
    # )

In [89]:
n = 4
def start_attention_tile_prob():
    video_path = r'C:\Users\es25591\Workspace\360dataset\content\saliency\pacman_saliency.mp4'
    cap = cv2.VideoCapture(video_path)

    ret_f, f = cap.read()
    if not ret_f:
        return
    att = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)    
    
    def split_into_tiles(arr, n):
        h, w = arr.shape
        tile_h, tile_w = h // n, w // n
        tiles = []
        for i in range(n):
            for j in range(n):
                y0, y1 = i * tile_h, (i + 1) * tile_h
                x0, x1 = j * tile_w, (j + 1) * tile_w
                tiles.append(arr[y0:y1, x0:x1])
        return np.array(tiles)

    tiles = split_into_tiles(att, n)

    total_attention = np.sum(att)

    att_prob = [
        float(np.sum(tile) / total_attention) for tile in tiles
    ]
    
    return att_prob

In [90]:
# CPT parameters (typical values from literature; adjustable)
alpha = 0.88  # gain curvature
beta = 0.88   # loss curvature
lam = 2.25    # loss aversion (losses weighted more)
prelec_gamma = 0.65  # probability weighting parameter (Prelec)
n = 4
num_tiles = n*n
class CPTPrefetchEnvSimple(gym.Env):
    def __init__(
        self, 
        num_tiles: int = num_tiles,
        user_tiles: np.ndarray = user_tiles,
        n_users: int = 1,
        cache_capacity: int = 10,
    ):
        super(CPTPrefetchEnvSimple, self).__init__()
        self.action_space = spaces.Discrete(2)  # Example: two discrete actions
        self.observation_space = spaces.Box(
            low=0, 
            high=1, 
            shape=(4,), 
            dtype=np.float32
        )  # Example: 4D continuous observation


        self.capacity = self.cache_capacity = cache_capacity
        self.sizes = np.random.randint(1, 4, size=num_tiles) 

        self.gain_if_prefetched = 1.0
        self.loss_if_not_prefetched = -2.0

        self.gamma = 0.7
        self.alpha = 0.88
        self.beta = 0.88
        self.lam = 2.25

        self.step_count = 0
        self.user_tiles = user_tiles

        # self.current_p = np.zeros(num_tiles, dtype=float)
        self.current_p = start_attention_tile_prob()
        print(self.current_p)


    def update_current_p(self) -> np.ndarray:
        p = np.zeros(num_tiles, dtype=int)

        for tile in self.user_tiles[self.step_count]:
            tx, ty = tile[0] - 1, tile[1] - 1
            p[tx + ty*n] = 1

        return p

    def reset(self):
        self.state = np.random.rand(4)  # Example initial state
        return self.state

    def step(self, action):
        self.state = np.random.rand(4)
        reward     = 0.0
        done       = np.random.rand() > 0.95
        info       = {}

        n_prefetch = int(action.sum())
        info = {}

        # Enforce capacity constraint and compute penalty if exceeded
        selected = np.asarray(np.where(action == 1)[0], dtype=int)
        if selected.size > 0 and n_prefetch > self.capacity:
            to_drop = int(n_prefetch - self.capacity)
            priorities = np.asarray(self.current_p)[selected]  # priorities aligned with selected
            order = np.argsort(priorities, kind='stable')      # indices into selected, ascending priorities
            sel_sorted = selected[order]
            drop = sel_sorted[:to_drop]
            action[drop] = 0
            info["clipped_dropped_indices"] = drop.tolist()
            penalty = -0.5 * to_drop
            reward += penalty

        reward += self.compute_expected_cpt_reward(action)

        return self.state, reward, done, info

    def compute_expected_cpt_reward(self, action: np.ndarray) -> float:
        expected_reward = 0.0
        for tile in range(num_tiles):
            p = float(self.current_p[tile])
            w = self.prelec_w(p)
            if action[tile] == 1:
                expected_outcome = p * self.gain_if_prefetched
            else:
                expected_outcome = p * self.loss_if_not_prefetched
            
            v = self.cpt_value(expected_outcome, alpha=self.alpha, beta=self.beta, lam=self.lam)
            expected_reward += w * v

        return float(expected_reward)

    def u_gain(self, x):
        return x**alpha

    def u_loss(self, x):
        return -lam * ((-x)**beta)
    
    def prelec_w(self, p: float, alpha: float = 0.65) -> float:
        if p <= 0.0: return 0.0
        if p >= 1.0: return 1.0
        return math.exp(-((-math.log(p)) ** alpha))

    def cpt_value_function(
        self,
        x: float, 
        alpha: float = 0.88, 
        beta: float = None, 
        lam: float = 2.25, 
        x0: float = 0.0
    ) -> float:
        if beta is None:
            beta = alpha
        diff = x - x0
        if diff >= 0:
            return diff ** alpha
        else:
            return -lam * ((-diff) ** beta)

    def cpt_value(self, x: float, alpha: float = 0.88, beta: float = 0.88, lam: float = 2.25) -> float:
        if x >= 0:
            return x ** alpha
        else:
            return -lam * ((-x) ** beta)

In [91]:
def map_user_tiles_to_action(step) -> np.ndarray:
    action = np.zeros(num_tiles, dtype=int)
    for tile in user_tiles[step]:
        tx, ty = tile[0] - 1, tile[1] - 1
        action[tx + ty*n] = 1
    return action

if __name__ == '__main__':
    env = CPTPrefetchEnvSimple(
        num_tiles=num_tiles,
        user_tiles=user_tiles,
        n_users=1,
        cache_capacity=50,
    )

    obs = env.reset()
    done = False
    total_reward = 0.0

    for step in range(60):
        action = map_user_tiles_to_action(step)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        print(f"Obs: {obs}, Reward: {reward}, Done: {done}")

    print(f"Total reward: {total_reward}")


[0.0008343041140910125, 0.02723348648278134, 0.02407229956387877, 0.0005747917525986161, 0.0006949913653000896, 0.2811378791377939, 0.1869168340207505, 0.0007377111462561423, 0.0008346325665210689, 0.23975113285164923, 0.19081337002375262, 0.000784693820453352, 0.0005634636379364598, 0.023069729968704843, 0.021688440744976792, 0.0002922388025552453]
Obs: [0.86783995 0.30139536 0.11279525 0.67878534], Reward: -0.7447469694904795, Done: False
Obs: [0.02011653 0.57720202 0.06502369 0.99648125], Reward: -0.7447469694904795, Done: False
Obs: [0.17588261 0.54859992 0.24876559 0.74500195], Reward: -0.7447469694904795, Done: True
Obs: [0.0936979  0.26781141 0.61545949 0.95084398], Reward: -0.7447469694904795, Done: False
Obs: [0.0908075  0.56770901 0.78860066 0.40292094], Reward: -0.7447469694904795, Done: False
Obs: [0.8986805  0.47242972 0.32114409 0.17230937], Reward: -0.7447469694904795, Done: True
Obs: [0.54596124 0.02032327 0.43671854 0.90341438], Reward: -0.7447469694904795, Done: False